Background for those reading: http://nlp.stanford.edu/projects/glove/

## Import Libraries and Data

In [7]:
import spacy
import pandas as pd
import numpy as np
import re
from sklearn.decomposition import PCA
#For counting of tokens
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words = 'english', max_df = 2)
#Load english corpus
en_nlp = spacy.load('en')
#Add additional stopwords
more_stopwords = ['ziisha', 'ziidisha', 'zidsisha',
       'zidishq', 'zidishia', 'zidishi', 'zidisher', 'zidishans',
       'zidisaha', 'zidisa', 'zidihsa', 'zididsha', 'zididisha', 'zididha',
       'zidiasha', 'ziddisha', 'zdisha', 'loans', 'loan', 'thank', 'you', 
        'business', 'funding', 'fund']
for word in more_stopwords:
    en_nlp.vocab[word].is_stop = True
#Load data from Stefan
dat = pd.read_csv('Regression_Dataset_20161221(na).csv')
#so you can see more columns
pd.options.display.max_columns = 500
pd.options.display.max_rows = 200

%matplotlib inline

## Define Helper Functions

In [31]:
def get_top_tokens(pandas_series, n=200):
    """
    gives you a count of most common tokens in text field
    """
    
    vect.fit(pandas_series.dropna())
    counts = list(vect.vocabulary_.items())
    df = pd.DataFrame(counts, columns=['token', 'count']).set_index('token').sort_values('count', ascending=False)
    return df.head(n)


def clean_text(text_string):
    """ 
    remove punctuation, convert to lowercase and remove stop words
    """
    doc =  en_nlp(text_string)
    return(' '.join([token.text.lower() for token in doc if token.is_stop == False and token.is_punct ==False]))


def vectorize_text(text_string, clean=True):
    """
    return GloVe vectors for words, document vector is the average of the word vectors
    reference: http://nlp.stanford.edu/projects/glove/
    """
    if type(text_string) != str:
        return np.zeros(300)  # return array of 300 zeros as built-in GloVe embedding has 300 dimensions
    
    if clean:
        text_string = clean_text(text_string)
        
    doc =  en_nlp(text_string)
    return np.round(doc.vector, 3) #reduce size of datatset by reducing significant digits

In [32]:
def attach_vectorized_column(df, column_name_list, clean=True):
    """
    append vectorized GloVe columns to dataframe
    
    warning: will add 300 columns for each text field
    """
    dataframes = []
    dataframes.append(df)
    
    for column_name in column_name_list:
        print('parsing {}'.format(column_name))
        
        col_names = [column_name+'_txtvec_'+str(x) for x in range(1, 301)]
        
        vecdf = pd.DataFrame.from_records((df[column_name].
        apply(lambda x: vectorize_text(x, clean=clean))),
        columns = col_names)
        
        dataframes.append(vecdf)
    
    return pd.concat(dataframes, axis = 1)

In [33]:
def attach_vectorized_column_summed(df, column_name_list, clean=True):
    """
    Sum all the vectors of the text fields as form of dimensionality reduction
    
    will only add 300 features summarizing all text, not 300 for each text field.  
    """
    vecs = []
    new_col_names = ['sum_txtvec_'+str(x) for x in range(1, 301)]

    for column_name in column_name_list:
        print('parsing {}'.format(column_name))
        vecs.append(df[column_name].apply(lambda x: vectorize_text(x, clean=clean)))
    
    combined_vecdf = pd.DataFrame.from_records(sum(vecs), 
                                               columns = new_col_names)
    
    return pd.concat([df, combined_vecdf], axis = 1)

# Preview Dataset

#### See top tokens in proposal field

In [6]:
get_top_tokens(dat.proposal_field, 20)

,count
token,
ön,18308
être,18307
été,18306
énorme,18305
élevage,18304
électricité,18303
écouler,18302
zte,18301
zorgho,18300


# Vectorize Text Fields

Using this technique called Glove to vectorize text:  http://nlp.stanford.edu/projects/glove/

Using this library:  https://spacy.io/  , where GloVe has been pretrained on the Common Crawl corpus

#### Warning:  computing the vectors can take 15-30 minutes, and is fairly memory hungry; if you are playing with this maybe just skip this part and load from csv

In [7]:
df = attach_vectorized_column(dat, ['about_me_field', 'about_business_field', 'proposal_field'])

parsing about_me_field
parsing about_business_field
parsing proposal_field


**Preview dataset with vectorized columns**

In [9]:
df.head()

,id,borrower_id,usd_amount,len_proposal,len_about_me,len_about_business,len_address,missing_natl_id,missing_referred_by,application_time,default_flag,fraud_flag,nonfraud_default,friends_count,country_id,name,category_id,invited_flag,made_pmts,missed_pmts,sift_science_score,english_flag,business_years,usd_installment_amount,reserve_fee_pct,sift_labeled_bad,prior_loans,country_internet_users,country_life_expectancy,country_literacy,country_gdppc,about_me_field,about_business_field,proposal_field,about_me_field_txtvec_1,about_me_field_txtvec_2,about_me_field_txtvec_3,about_me_field_txtvec_4,about_me_field_txtvec_5,about_me_field_txtvec_6,about_me_field_txtvec_7,about_me_field_txtvec_8,about_me_field_txtvec_9,about_me_field_txtvec_10,about_me_field_txtvec_11,about_me_field_txtvec_12,about_me_field_txtvec_13,about_me_field_txtvec_14,about_me_field_txtvec_15,about_me_field_txtvec_16,about_me_field_txtvec_17,about_me_field_txtvec_18,about_me_field_txtvec_19,about_me_field_txtvec_20,about_me_field_txtvec_21,about_me_field_txtvec_22,about_me_field_txtvec_23,about_me_field_txtvec_24,about_me_field_txtvec_25,about_me_field_txtvec_26,about_me_field_txtvec_27,about_me_field_txtvec_28,about_me_field_txtvec_29,about_me_field_txtvec_30,about_me_field_txtvec_31,about_me_field_txtvec_32,about_me_field_txtvec_33,about_me_field_txtvec_34,about_me_field_txtvec_35,about_me_field_txtvec_36,about_me_field_txtvec_37,about_me_field_txtvec_38,about_me_field_txtvec_39,about_me_field_txtvec_40,about_me_field_txtvec_41,about_me_field_txtvec_42,about_me_field_txtvec_43,about_me_field_txtvec_44,about_me_field_txtvec_45,about_me_field_txtvec_46,about_me_field_txtvec_47,about_me_field_txtvec_48,about_me_field_txtvec_49,about_me_field_txtvec_50,about_me_field_txtvec_51,about_me_field_txtvec_52,about_me_field_txtvec_53,about_me_field_txtvec_54,about_me_field_txtvec_55,about_me_field_txtvec_56,about_me_field_txtvec_57,about_me_field_txtvec_58,about_me_field_txtvec_59,about_me_field_txtvec_60,about_me_field_txtvec_61,about_me_field_txtvec_62,about_me_field_txtvec_63,about_me_field_txtvec_64,about_me_field_txtvec_65,about_me_field_txtvec_66,about_me_field_txtvec_67,about_me_field_txtvec_68,about_me_field_txtvec_69,about_me_field_txtvec_70,about_me_field_txtvec_71,about_me_field_txtvec_72,about_me_field_txtvec_73,about_me_field_txtvec_74,about_me_field_txtvec_75,about_me_field_txtvec_76,about_me_field_txtvec_77,about_me_field_txtvec_78,about_me_field_txtvec_79,about_me_field_txtvec_80,about_me_field_txtvec_81,about_me_field_txtvec_82,about_me_field_txtvec_83,about_me_field_txtvec_84,about_me_field_txtvec_85,about_me_field_txtvec_86,about_me_field_txtvec_87,about_me_field_txtvec_88,about_me_field_txtvec_89,about_me_field_txtvec_90,about_me_field_txtvec_91,about_me_field_txtvec_92,about_me_field_txtvec_93,about_me_field_txtvec_94,about_me_field_txtvec_95,about_me_field_txtvec_96,about_me_field_txtvec_97,about_me_field_txtvec_98,about_me_field_txtvec_99,about_me_field_txtvec_100,about_me_field_txtvec_101,about_me_field_txtvec_102,about_me_field_txtvec_103,about_me_field_txtvec_104,about_me_field_txtvec_105,about_me_field_txtvec_106,about_me_field_txtvec_107,about_me_field_txtvec_108,about_me_field_txtvec_109,about_me_field_txtvec_110,about_me_field_txtvec_111,about_me_field_txtvec_112,about_me_field_txtvec_113,about_me_field_txtvec_114,about_me_field_txtvec_115,about_me_field_txtvec_116,about_me_field_txtvec_117,about_me_field_txtvec_118,about_me_field_txtvec_119,about_me_field_txtvec_120,about_me_field_txtvec_121,about_me_field_txtvec_122,about_me_field_txtvec_123,about_me_field_txtvec_124,about_me_field_txtvec_125,about_me_field_txtvec_126,about_me_field_txtvec_127,about_me_field_txtvec_128,about_me_field_txtvec_129,about_me_field_txtvec_130,about_me_field_txtvec_131,about_me_field_txtvec_132,about_me_field_txtvec_133,about_me_field_txtvec_134,about_me_field_txtvec_135,about_me_field_txtvec_136,about_me_field_txtvec_137,about_me_field_txtvec_138,about_me_field_txtvec

In [34]:
df_summed = attach_vectorized_column_summed(dat, ['about_me_field', 'about_business_field', 'proposal_field'])
df_summed.head()

parsing about_me_field
parsing about_business_field
parsing proposal_field


,id,borrower_id,usd_amount,len_proposal,len_about_me,len_about_business,len_address,missing_natl_id,missing_referred_by,application_time,default_flag,fraud_flag,nonfraud_default,friends_count,country_id,name,category_id,invited_flag,made_pmts,missed_pmts,sift_science_score,english_flag,business_years,usd_installment_amount,reserve_fee_pct,sift_labeled_bad,prior_loans,country_internet_users,country_life_expectancy,country_literacy,country_gdppc,about_me_field,about_business_field,proposal_field,sum_txtvec_1,sum_txtvec_2,sum_txtvec_3,sum_txtvec_4,sum_txtvec_5,sum_txtvec_6,sum_txtvec_7,sum_txtvec_8,sum_txtvec_9,sum_txtvec_10,sum_txtvec_11,sum_txtvec_12,sum_txtvec_13,sum_txtvec_14,sum_txtvec_15,sum_txtvec_16,sum_txtvec_17,sum_txtvec_18,sum_txtvec_19,sum_txtvec_20,sum_txtvec_21,sum_txtvec_22,sum_txtvec_23,sum_txtvec_24,sum_txtvec_25,sum_txtvec_26,sum_txtvec_27,sum_txtvec_28,sum_txtvec_29,sum_txtvec_30,sum_txtvec_31,sum_txtvec_32,sum_txtvec_33,sum_txtvec_34,sum_txtvec_35,sum_txtvec_36,sum_txtvec_37,sum_txtvec_38,sum_txtvec_39,sum_txtvec_40,sum_txtvec_41,sum_txtvec_42,sum_txtvec_43,sum_txtvec_44,sum_txtvec_45,sum_txtvec_46,sum_txtvec_47,sum_txtvec_48,sum_txtvec_49,sum_txtvec_50,sum_txtvec_51,sum_txtvec_52,sum_txtvec_53,sum_txtvec_54,sum_txtvec_55,sum_txtvec_56,sum_txtvec_57,sum_txtvec_58,sum_txtvec_59,sum_txtvec_60,sum_txtvec_61,sum_txtvec_62,sum_txtvec_63,sum_txtvec_64,sum_txtvec_65,sum_txtvec_66,sum_txtvec_67,sum_txtvec_68,sum_txtvec_69,sum_txtvec_70,sum_txtvec_71,sum_txtvec_72,sum_txtvec_73,sum_txtvec_74,sum_txtvec_75,sum_txtvec_76,sum_txtvec_77,sum_txtvec_78,sum_txtvec_79,sum_txtvec_80,sum_txtvec_81,sum_txtvec_82,sum_txtvec_83,sum_txtvec_84,sum_txtvec_85,sum_txtvec_86,sum_txtvec_87,sum_txtvec_88,sum_txtvec_89,sum_txtvec_90,sum_txtvec_91,sum_txtvec_92,sum_txtvec_93,sum_txtvec_94,sum_txtvec_95,sum_txtvec_96,sum_txtvec_97,sum_txtvec_98,sum_txtvec_99,sum_txtvec_100,sum_txtvec_101,sum_txtvec_102,sum_txtvec_103,sum_txtvec_104,sum_txtvec_105,sum_txtvec_106,sum_txtvec_107,sum_txtvec_108,sum_txtvec_109,sum_txtvec_110,sum_txtvec_111,sum_txtvec_112,sum_txtvec_113,sum_txtvec_114,sum_txtvec_115,sum_txtvec_116,sum_txtvec_117,sum_txtvec_118,sum_txtvec_119,sum_txtvec_120,sum_txtvec_121,sum_txtvec_122,sum_txtvec_123,sum_txtvec_124,sum_txtvec_125,sum_txtvec_126,sum_txtvec_127,sum_txtvec_128,sum_txtvec_129,sum_txtvec_130,sum_txtvec_131,sum_txtvec_132,sum_txtvec_133,sum_txtvec_134,sum_txtvec_135,sum_txtvec_136,sum_txtvec_137,sum_txtvec_138,sum_txtvec_139,sum_txtvec_140,sum_txtvec_141,sum_txtvec_142,sum_txtvec_143,sum_txtvec_144,sum_txtvec_145,sum_txtvec_146,sum_txtvec_147,sum_txtvec_148,sum_txtvec_149,sum_txtvec_150,sum_txtvec_151,sum_txtvec_152,sum_txtvec_153,sum_txtvec_154,sum_txtvec_155,sum_txtvec_156,sum_txtvec_157,sum_txtvec_158,sum_txtvec_159,sum_txtvec_160,sum_txtvec_161,sum_txtvec_162,sum_txtvec_163,sum_txtvec_164,sum_txtvec_165,sum_txtvec_166,sum_txtvec_167,sum_txtvec_168,sum_txtvec_169,sum_txtvec_170,sum_txtvec_171,sum_txtvec_172,sum_txtvec_173,sum_txtvec_174,sum_txtvec_175,sum_txtvec_176,sum_txtvec_177,sum_txtvec_178,sum_txtvec_179,sum_txtvec_180,sum_txtvec_181,sum_txtvec_182,sum_txtvec_183,sum_txtvec_184,sum_txtvec_185,sum_txtvec_186,sum_txtvec_187,sum_txtvec_188,sum_txtvec_189,sum_txtvec_190,sum_txtvec_191,sum_txtvec_192,sum_txtvec_193,sum_txtvec_194,sum_txtvec_195,sum_txtvec_196,sum_txtvec_197,sum_txtvec_198,sum_txtvec_199,sum_txtvec_200,sum_txtvec_201,sum_txtvec_202,sum_txtvec_203,sum_txtvec_204,sum_txtvec_205,sum_txtvec_206,sum_txtvec_207,sum_txtvec_208,sum_txtvec_209,sum_txtvec_210,sum_txtvec_211,sum_txtvec_212,sum_txtvec_213,sum_txtvec_214,sum_txtvec_215,sum_txtvec_216,sum_txtvec_217,sum_txtvec_218,sum_txtvec_219,sum_txtvec_220,sum_txtvec_221,sum_txtvec_222,sum_txtvec_223,sum_txtvec_224,sum_txtvec_225,sum_txtvec_226,sum_txtvec_227,sum_txtvec_228,sum_txtvec_229,sum_txtvec_230,sum_txtvec_231,sum_txtvec_232,sum_txtvec_233,sum_txtvec_234,sum_txtvec_235,sum_txtvec_236,sum_txtvec_237,sum_txtvec_238,sum_txtvec_239,sum_txtvec_240,sum_txtve

# Prepare data for DataRobot and Upload

helper function that is not used until later

In [44]:
import datarobot as dr

def send_to_datarobot(df, proj_name):
    """
    helper function that takes data and uploads to DataRobot
    """
    df = df.copy()
    
    #drop irrelevant columns
    cols_to_drop = ['missed_pmts', 'nonfraud_default', 'sift_labeled_bad', 'sift_science_score', 
                'made_pmts', 'fraud_flag', 'reserve_fee_pct','country_id']
    final_df = df.drop(cols_to_drop, axis=1)
    #Combine the ids to obtain a unique user id
    final_df['unique_id'] = final_df.borrower_id.astype(str) +'__'+ final_df.id.astype(str)
    final_df.drop(['borrower_id', 'id'], axis = 1, inplace=True)
    
    #save file to disk, with compression
    file_path = '/Users/hamelhusain/Google Drive/Team Shared Folder - Zidisha/DR_'+proj_name+'.gz'
    print('Saving {}'.format(file_path))
    final_df.to_csv(file_path, compression = 'gzip', index=False)
    
    print('Beginning DR Upload')
    proj = dr.Project.create(file_path, project_name=proj_name)
    proj.set_target(target = 'default_flag', 
                metric = 'AUC', 
                #mode = dr.AUTOPILOT_MODE.QUICK,
                worker_count = 30)
    
    return proj

### Experiment 1:  Raw Text + Embeddings
https://app.datarobot.com/projects/58660cbbc808916efd0947d6/models



#### Drop irrelevant columns, check size of data

In [37]:
cols_to_drop = ['missed_pmts', 'nonfraud_default', 'sift_labeled_bad', 'sift_science_score', 
                'made_pmts', 'fraud_flag', 'reserve_fee_pct','country_id']
final_df = df.drop(cols_to_drop, axis=1)

In [38]:
#Combine the ids to obtain a unique user id
final_df['unique_id'] = final_df.borrower_id.astype(str) +'__'+ final_df.id.astype(str)
final_df.drop(['borrower_id', 'id'], axis = 1, inplace=True)

#### Save Data

In [47]:
file_path = '/Users/hamelhusain/Google Drive/Team Shared Folder - Zidisha/Data/TextVec_Experiment.gz'
final_df.to_csv(file_path, compression = 'gzip', index=False)

#### Upload to DataRobot

In [48]:
proj = dr.Project.create(file_path, project_name='Text Vector With Text')

In [50]:
proj.set_target(target = 'default_flag', 
                metric = 'AUC', 
                mode = dr.AUTOPILOT_MODE.QUICK,
                worker_count = 15)

ClientError: 422 client error: {'message': 'Target default_flag was already selected.'}

### Experiment 2: Remove Original Text Fields and Only Keep Embeddings

https://app.datarobot.com/projects/58660cbbc808916efd0947d6/models

In [112]:
filepath2 = '/Users/hamelhusain/Google Drive/Team Shared Folder - Zidisha/Data/TextVec_Experiment_NOTEXT.gz'
final_df_notext = final_df.drop(['about_me_field', 'about_business_field', 'proposal_field'], axis =1)
final_df_notext.to_csv(filepath2, compression = 'gzip', index=False)

In [125]:
proj2 = dr.Project.create(filepath2, project_name='Text Vector No Text')
proj2.set_target(target = 'default_flag', 
                metric = 'AUC', 
                worker_count = 10)

Project(Text Vector No Text)

In [ ]:
proj.get_leaderboard_ui_permalink()

### Experiment 3: Cleaned Text, Summed Vectors From Three Fields

https://app.datarobot.com/manage-projects

cleaned text by removing stopwords, punctuation and converting to lower-case
Will have these three different feature lists:

1. raw text only
2. summed vectors without raw text
3. both


In [ ]:
proj3 = send_to_datarobot(df_summed, 'Text_SummedVec')

Saving /Users/hamelhusain/Google Drive/Team Shared Folder - Zidisha/DR_Text_SummedVec.gz
Beginning DR Upload


# Appendix (Things I Tried But Didn't Work)

### Experiment - PCA on Word Emebeddings -> reduced from 300 to 25 dimensions

https://app.datarobot.com/projects/58660cbbc808916efd0947d6/models

25 dimensions captured 90% of the variance.  

didn't work well, noticable loss in accuracy 



In [115]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
df = final_df_notext.copy()
train_df, test_df, _, _ = train_test_split(df, df.default_flag, test_size=0.4, train_size=.6, random_state=1234)
val_df, holdout_df, _, _ = train_test_split(test_df, test_df.default_flag, 
                                            test_size = .5, train_size =.5, random_state=54)
val_df['Part'] = 'v'
train_df['Part'] = 't'
holdout_df['Part'] = 'h'
df = pd.concat([train_df, val_df, holdout_df])

/Users/hamelhusain/anaconda/envs/drpy3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/hamelhusain/anaconda/envs/drpy3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/hamelhusain/anaconda/envs/drpy3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [126]:
def embeddingPCA(df, column_names):
    df_temp = df.copy()
    
    pca = PCA(n_components=100)
    transformed_dfs = []
    
    for column_name in column_names:
        #compute names to drop and new names after pca
        names = [column_name+'_txtvec_'+str(x) for x in range(1, 301)]
        new_names = [column_name+'_pcavec_'+str(x) for x in range(1, 101)]
        
        #perform pca on training set and apply to entire dataset
        pca.fit(df_temp[df_temp.Part == 't'][names])
        pca_df = pd.DataFrame(pca.transform(df_temp[names]), columns=new_names)
        
        #collect components
        transformed_dfs.append(pca_df)
        #drop original vectors
        df_temp.drop(names, axis =1, inplace = True)

    #assemble all pieces
    return(pd.concat([df_temp] + transformed_dfs, axis = 1))

In [127]:
compressed_vec_df = embeddingPCA(df, ['about_me_field', 'about_business_field', 'proposal_field'])

In [128]:
compressed_vec_df.head()

,usd_amount,len_proposal,len_about_me,len_about_business,len_address,missing_natl_id,missing_referred_by,application_time,default_flag,friends_count,name,category_id,invited_flag,english_flag,business_years,usd_installment_amount,prior_loans,country_internet_users,country_life_expectancy,country_literacy,country_gdppc,unique_id,Part,about_me_field_pcavec_1,about_me_field_pcavec_2,about_me_field_pcavec_3,about_me_field_pcavec_4,about_me_field_pcavec_5,about_me_field_pcavec_6,about_me_field_pcavec_7,about_me_field_pcavec_8,about_me_field_pcavec_9,about_me_field_pcavec_10,about_me_field_pcavec_11,about_me_field_pcavec_12,about_me_field_pcavec_13,about_me_field_pcavec_14,about_me_field_pcavec_15,about_me_field_pcavec_16,about_me_field_pcavec_17,about_me_field_pcavec_18,about_me_field_pcavec_19,about_me_field_pcavec_20,about_me_field_pcavec_21,about_me_field_pcavec_22,about_me_field_pcavec_23,about_me_field_pcavec_24,about_me_field_pcavec_25,about_me_field_pcavec_26,about_me_field_pcavec_27,about_me_field_pcavec_28,about_me_field_pcavec_29,about_me_field_pcavec_30,about_me_field_pcavec_31,about_me_field_pcavec_32,about_me_field_pcavec_33,about_me_field_pcavec_34,about_me_field_pcavec_35,about_me_field_pcavec_36,about_me_field_pcavec_37,about_me_field_pcavec_38,about_me_field_pcavec_39,about_me_field_pcavec_40,about_me_field_pcavec_41,about_me_field_pcavec_42,about_me_field_pcavec_43,about_me_field_pcavec_44,about_me_field_pcavec_45,about_me_field_pcavec_46,about_me_field_pcavec_47,about_me_field_pcavec_48,about_me_field_pcavec_49,about_me_field_pcavec_50,about_me_field_pcavec_51,about_me_field_pcavec_52,about_me_field_pcavec_53,about_me_field_pcavec_54,about_me_field_pcavec_55,about_me_field_pcavec_56,about_me_field_pcavec_57,about_me_field_pcavec_58,about_me_field_pcavec_59,about_me_field_pcavec_60,about_me_field_pcavec_61,about_me_field_pcavec_62,about_me_field_pcavec_63,about_me_field_pcavec_64,about_me_field_pcavec_65,about_me_field_pcavec_66,about_me_field_pcavec_67,about_me_field_pcavec_68,about_me_field_pcavec_69,about_me_field_pcavec_70,about_me_field_pcavec_71,about_me_field_pcavec_72,about_me_field_pcavec_73,about_me_field_pcavec_74,about_me_field_pcavec_75,about_me_field_pcavec_76,about_me_field_pcavec_77,about_me_field_pcavec_78,about_me_field_pcavec_79,about_me_field_pcavec_80,about_me_field_pcavec_81,about_me_field_pcavec_82,about_me_field_pcavec_83,about_me_field_pcavec_84,about_me_field_pcavec_85,about_me_field_pcavec_86,about_me_field_pcavec_87,about_me_field_pcavec_88,about_me_field_pcavec_89,about_me_field_pcavec_90,about_me_field_pcavec_91,about_me_field_pcavec_92,about_me_field_pcavec_93,about_me_field_pcavec_94,about_me_field_pcavec_95,about_me_field_pcavec_96,about_me_field_pcavec_97,about_me_field_pcavec_98,about_me_field_pcavec_99,about_me_field_pcavec_100,about_business_field_pcavec_1,about_business_field_pcavec_2,about_business_field_pcavec_3,about_business_field_pcavec_4,about_business_field_pcavec_5,about_business_field_pcavec_6,about_business_field_pcavec_7,about_business_field_pcavec_8,about_business_field_pcavec_9,about_business_field_pcavec_10,about_business_field_pcavec_11,about_business_field_pcavec_12,about_business_field_pcavec_13,about_business_field_pcavec_14,about_business_field_pcavec_15,about_business_field_pcavec_16,about_business_field_pcavec_17,about_business_field_pcavec_18,about_business_field_pcavec_19,about_business_field_pcavec_20,about_business_field_pcavec_21,about_business_field_pcavec_22,about_business_field_pcavec_23,about_business_field_pcavec_24,about_business_field_pcavec_25,about_business_field_pcavec_26,about_business_field_pcavec_27,about_business_field_pcavec_28,about_business_field_pcavec_29,about_business_field_pcavec_30,about_business_field_pcavec_31,about_business_field_pcavec_32,about_business_field_pcavec_33,about_business_field_pcavec_34,about_business_field_pcavec_35,about_business_field_pcavec_36,about_business_field_pcavec_37,about_business_field_pcavec_38,abou

In [129]:
tvh = dr.UserTVH(user_partition_col='Part', training_level='t', validation_level='v', holdout_level='h')
proj3 = dr.Project.create(compressed_vec_df, project_name='Text Vector PCA')

In [130]:
proj3.set_target(target = 'default_flag', 
                metric = 'AUC', 
                partitioning_method = tvh,
                mode = dr.AUTOPILOT_MODE.QUICK,
                worker_count = 10)

Project(Text Vector PCA)